<h1><center> Kaggle Competition: [New York City Taxi Fare Prediction](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction)</center></h1>

---
# 1. Data Ingestion & Data Cleaning

## 1.1 Data Ingestion

In [ ]:
import math
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
import subprocess
import seaborn as sns
sns.set()

import time
start_time = time.time()

In [ ]:
# Try to read just 5 rows to check if "data/train.csv" was downloaded:
try:
    train_df = pd.read_csv('data/train.csv', nrows=5)   # total number of rows: 55423857
    display(train_df.head())
except FileNotFoundError:
    print('>>> You must download train.csv from: https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/download/train.csv')
    exit(0)

#### Tricks to load the data and decrease memory footprint significantly

In [ ]:
# set column types to optimize memory usage
col_types = {
    'fare_amount': 'float32',
    'pickup_datetime': 'str', 
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'passenger_count': 'uint8'
}

new_cols = list(col_types.keys())

In [ ]:
def get_raw_dataframe(read_size=0):   
    # Method #1: read the entire DataFrame from the disk
    if (read_size == 0):
        df_list = [] # list to hold the batch dataframe
        for df_chunk in tqdm(pd.read_csv('data/train.csv', usecols=new_cols, dtype=col_types, chunksize=5000000)): # loads 5M rows each iteration
            df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0, 16)
            df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

            # Process each chunk of dataframe right here:
            # clean_data(), feature_engineer(),fit()

            # Or, append the chunk to list and merge all later
            df_list.append(df_chunk) 

        # merge all dataframes into one
        train_df = pd.concat(df_list)
        del df_list
        return train_df
        
    # Method #2: read a predetermined amount of data         
    train_df = pd.read_csv('data/train.csv', usecols=new_cols, dtype=col_types, nrows=read_size)
    train_df['pickup_datetime'] = train_df['pickup_datetime'].str.slice(0, 16)
    train_df['pickup_datetime'] = pd.to_datetime(train_df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    return train_df

## 1.2 Data Cleaning

In [ ]:
def remove_geo_outliers(df):
    # NY boundary
    boundary = { 
        'min_lng':-74.263242, 'min_lat':40.573143, 
        'max_lng':-72.986532, 'max_lat':41.709555 
    }
   
    df.loc[ ((df.pickup_longitude  >= boundary['min_lng'] ) & 
             (df.pickup_longitude  <= boundary['max_lng']) &
             (df.pickup_latitude   >= boundary['min_lat']) & 
             (df.pickup_latitude   <= boundary['max_lat']) &
             (df.dropoff_longitude >= boundary['min_lng']) & 
             (df.dropoff_longitude <= boundary['max_lng']) &
             (df.dropoff_latitude  >= boundary['min_lat']) & 
             (df.dropoff_latitude  <= boundary['max_lat'])),'outlier'] = 0
    
    df.loc[~((df.pickup_longitude  >= boundary['min_lng']) & 
             (df.pickup_longitude  <= boundary['max_lng']) &
             (df.pickup_latitude   >= boundary['min_lat']) & 
             (df.pickup_latitude   <= boundary['max_lat']) &
             (df.dropoff_longitude >= boundary['min_lng']) & 
             (df.dropoff_longitude <= boundary['max_lng']) &
             (df.dropoff_latitude  >= boundary['min_lat']) & 
             (df.dropoff_latitude  <= boundary['max_lat'])),'outlier'] = 1    
    
#     print("Outlier vs Non Outlier")
#     print(df['is_outlier_loc'].value_counts())

    df = df.loc[df['outlier'] == 0]    
    return df.drop(['outlier'], axis=1)

In [ ]:
def get_clean_dataframe(num_rows):
    train_df = get_raw_dataframe(num_rows)
    
    # remove NaNs and Negative & Free fares (keeping only the ones that cost more than 0)
    train_df.dropna(inplace=True, axis=0)
    train_df = train_df[train_df['fare_amount'] > 0]
#     train_df = train_df[(train_df['fare_amount'] >= 2) & (train_df['fare_amount'] <= 200)]

    # remove rows where lat/long are ZERO
    train_df = train_df[(train_df['pickup_latitude'] != 0) & (train_df['pickup_longitude'] != 0)]
    train_df = train_df[(train_df['dropoff_latitude'] != 0) & (train_df['dropoff_longitude'] != 0)]

    # handle rides with too many or too few passengers
    train_df = train_df[(train_df['passenger_count'] >= 0) & (train_df['passenger_count'] <= 7)]
    
    # remove any point beyond NYC border
    train_df = remove_geo_outliers(train_df)
        
    return train_df

---
# 2. Feature Engineering

In [ ]:
# Distance formula: https://www.movable-type.co.uk/scripts/latlong.html
def getDistance(lat1, lon1, lat2, lon2): 
    R = 6371e3
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    deltaPhi = math.radians(lat2-lat1)
    deltaLambda = math.radians(lon2-lon1)

    a = math.sin(deltaPhi/2) * math.sin(deltaPhi/2) + math.cos(phi1) * math.cos(phi2) * math.sin(deltaLambda/2) * math.sin(deltaLambda/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return round((R * c) / 1000, 3)    # returns distance in km

In [ ]:
# Add calendar related features: year, month, day, hour, day_of_week
def add_calendar(df):
    df['year'] = df.pickup_datetime.apply(lambda x: x.year)
    df['month'] = df.pickup_datetime.apply(lambda x: x.month)
    df['day'] = df.pickup_datetime.apply(lambda x: x.day)
    df['hour'] = df.pickup_datetime.apply(lambda x: x.hour)
    df['day_of_week'] = df.pickup_datetime.apply(lambda x: x.weekday())
    df['quarter'] = df.pickup_datetime.apply(lambda x: x.quarter)
    df['week_of_year'] = df.pickup_datetime.apply(lambda x: x.isocalendar()[1])        
    df['business_day'] = df.pickup_datetime.apply(lambda x: 1 if (x.weekday() >= 0 and x.weekday() <= 4) else 0)
    df['daytime'] = 0
    df.loc[(df.hour >= 8) & (df.hour < 20), 'daytime'] = 1 
    
    df['year'] = df['year'].astype('int16')
    df['month'] = df['month'].astype('int8')
    df['day'] = df['day'].astype('int8')
    df['hour'] = df['hour'].astype('int8')
    df['day_of_week'] = df['day_of_week'].astype('int8')    
    df['quarter'] = df['quarter'].astype('int8')
    df['week_of_year'] = df['week_of_year'].astype('int8')
    df['daytime'] = df['daytime'].astype('int8')
    df['business_day'] = df['business_day'].astype('int8')
    return df

In [ ]:
# Add distance in kilometers
def add_distance_km(df):
    df['distance_km'] = df.apply(lambda row: getDistance(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
    df['distance_km'] = df['distance_km'].astype('float32')

#     df['distance_km_round'] = df.distance_km.apply(lambda x: int(round(x)))
#     df['distance_km_round'] = df['distance_km_round'].astype('int16')
    return df

In [ ]:
def minkowski_distance(x1, x2, y1, y2, p):
    return ((abs(x2 - x1) ** p) + (abs(y2 - y1)) ** p) ** (1 / p)

def add_distances_mht_ecd(df):
    df['distance_mht'] = minkowski_distance(df['pickup_longitude'], df['dropoff_longitude'], df['pickup_latitude'], df['dropoff_latitude'], 1)
    df['distance_ecd'] = minkowski_distance(df['pickup_longitude'], df['dropoff_longitude'], df['pickup_latitude'], df['dropoff_latitude'], 2)    
    
    df['distance_mht'] = df['distance_mht'].astype('float32')
    df['distance_ecd'] = df['distance_ecd'].astype('float32')
    return df

In [ ]:
import math

def calc_compass_bearing(lat1, long1, lat2, long2):
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)

    diffLong = math.radians(long2 - long1)

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # atan2() returns values from -180° to + 180°
    # For a proper compass bearing, normalize the value as:
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360        
    return round(compass_bearing, 3)


def add_compass_bearing(df):
    df['trip_direction'] = df.apply(lambda row: calc_compass_bearing(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
    df['trip_direction'] = df['trip_direction'].astype('float32')
    return df

In [ ]:
# Add geographical cluster as feature 
from sklearn.cluster import KMeans
import copy

# When this function is called with 1 arg, the arg is the training dataset. 
# When called with both args, the 1st arg is the test and the 2nd is the training dataset.
def add_geo_cluster(df, train=pd.DataFrame()):
    kmeans_geo_model = KMeans(n_clusters=4) # 4 clusters, as indicated by the Elbow method
    
    # The test and training datasets are passed so they can be clustered together and the test gets proper labels.
    if (not train.empty):
        df['is_test'] = 1
        train['is_test'] = 0
        geo_df = pd.concat([df,train], axis=0, sort=True)[['pickup_longitude','pickup_latitude','is_test']]
    else:  
        geo_df = copy.deepcopy(df[['pickup_longitude','pickup_latitude']])
        
    kmeans_geo_model.fit(geo_df[['pickup_longitude','pickup_latitude']])        
    cluster_labels = kmeans_geo_model.predict(geo_df[['pickup_longitude','pickup_latitude']])    

    # At this point, cluster_labels might have all the labels for the test & train datasets,
    # and we can't simply do: df['geo_cluster'] = cluster_labels
    # because when this function is called with both params, df will just store the test data.
    # It's like trying to fit a big box into a smaller one.
    geo_df['geo_cluster'] = cluster_labels    # SettingWithCopyWarning: geo_df cannot be just a slice of df
    
    if (train.empty):
        df['geo_cluster'] = cluster_labels    
        return df 
        
    df['geo_cluster'] = geo_df[geo_df['is_test'] == 1].geo_cluster         
    train.drop('is_test', axis=1, inplace=True)
    df.drop('is_test', axis=1, inplace=True)  
    return df

In [ ]:
# Add geographical difference (ride_start - ride_end) as feature 
def add_geo_diff(df):
    df['lon_diff'] = df['dropoff_longitude'] - df['pickup_longitude']
    df['lat_diff'] = df['dropoff_latitude']  - df['pickup_latitude']
    return df

In [ ]:
# Add geographical coordinates rounded for less precision
def add_geo_coords_rounded(df):
    df['pickup_latitude_round']    = df['pickup_latitude'].apply(lambda x:round(x,3)).astype('float32')
    df['pickup_longitude_round']   = df['pickup_longitude'].apply(lambda x:round(x,3)).astype('float32')
    df['dropoff_latitude_round']   = df['dropoff_latitude'].apply(lambda x:round(x,3)).astype('float32')
    df['dropoff_longitude_round']  = df['dropoff_longitude'].apply(lambda x:round(x,3)).astype('float32')
    return df

In [ ]:
# Add features that define if a trip was to a particular airport OR neighbourhood
def setLocationSrcDst(df, boundary, srcName, dstName):
    df[srcName] = 0             
    df.loc[((df.pickup_latitude  >= boundary['min_lat']) & (df.pickup_latitude  <= boundary['max_lat']) &
            (df.pickup_longitude >= boundary['min_lng']) & (df.pickup_longitude <= boundary['max_lng'])), srcName] = 1    
    
    df[dstName] = 0
    df.loc[((df.dropoff_latitude  >= boundary['min_lat']) & (df.dropoff_latitude  <= boundary['max_lat']) &
            (df.dropoff_longitude >= boundary['min_lng']) & (df.dropoff_longitude <= boundary['max_lng'])), dstName] = 1
    
    df[srcName] = df[srcName].astype('int8')
    df[dstName] = df[dstName].astype('int8')
    return df


def add_airports(df):
    JFK = { 'min_lng': -73.8352, 'min_lat': 40.6195, 'max_lng': -73.7401, 'max_lat': 40.6659 }        
    EWR = { 'min_lng': -74.1925, 'min_lat': 40.6700, 'max_lng': -74.1531, 'max_lat': 40.7081 }
    LGA = { 'min_lng': -73.8895, 'min_lat': 40.7664, 'max_lng': -73.8550, 'max_lat': 40.7931 }
    
    df = setLocationSrcDst(df, JFK, 'jfk_src', 'jfk_dst')        
    df = setLocationSrcDst(df, EWR, 'ewr_src', 'ewr_dst')        
    df = setLocationSrcDst(df, LGA, 'lga_src', 'lga_dst')    
    return df   


def add_locations(df):
    manhattan =     { 'min_lng': -74.0479, 'min_lat': 40.6829, 'max_lng': -73.9067, 'max_lat': 40.8820 }                
    queens =        { 'min_lng': -73.9630, 'min_lat': 40.5431, 'max_lng': -73.7004, 'max_lat': 40.8007 }
    brooklyn =      { 'min_lng': -74.0421, 'min_lat': 40.5707, 'max_lng': -73.8334, 'max_lat': 40.7395 }
    bronx =         { 'min_lng': -73.9339, 'min_lat': 40.7855, 'max_lng':-73.7654,  'max_lat': 40.9176 }
    staten_island = { 'min_lng': -74.2558, 'min_lat': 40.4960, 'max_lng': -74.0522, 'max_lat': 40.6490 }
    
    df = setLocationSrcDst(df, manhattan, 'man_src', 'man_dst')
    df = setLocationSrcDst(df, queens, 'qns_src', 'qns_dst')
    df = setLocationSrcDst(df, brooklyn, 'bkny_src', 'bkny_dst')
    df = setLocationSrcDst(df, bronx, 'bx_src', 'bx_dst')
    df = setLocationSrcDst(df, staten_island, 'si_src', 'si_dst')
    
    return df

In [ ]:
def add_hour_statistics(df):   
    # peak hour only exists on business days
    df['peak_hour'] = np.where((df['hour'] >= 16) & (df['hour'] <= 20) & (df['business_day'] == 1), 1, 0)
        
    # there is a daily 50-cent surcharge from 8pm to 6am
    df['night_hour'] = np.where((df['hour'] >= 20) | (df['hour'] <= 6) , 1, 0)
    
    df['peak_hour'] = df['peak_hour'].astype('int8')
    df['night_hour'] = df['night_hour'].astype('int8')
    return df

In [ ]:
# Add ratio for (rides_per_hour and rides_per_day) (only works for the training set)
def add_ride_statistics(df):    
    tmp_hour = df.groupby(['year','month','day','hour']).agg({'fare_amount' : [np.size]}).reset_index()
    tmp_hour.columns = ['year','month','day','hour','rides_per_hour']

    tmp_day = df.groupby(['year','month','day']).agg({'fare_amount' : [np.size]}).reset_index()
    tmp_day.columns = ['year','month','day','rides_per_day']
            
    df = pd.merge(df, tmp_hour, how='left', on=['year','month','day','hour'])
    df = pd.merge(df, tmp_day, how='left', on=['year','month','day'])
        
    df['rides_per_hour'] = df['rides_per_hour'].astype('int16')
    df['rides_per_day'] = df['rides_per_day'].astype('int16')
    return df

In [ ]:
# Add mean/median/min/max fare (only works well for the training/test set, not for the Kaggle scoring system)
def add_fare_statistics(df):
    tmp = df.groupby(['year','month','day_of_week','hour','distance_km_round','taxis_per_hour']).agg({'fare_amount' : [np.mean,np.median,np.min,np.max]}).reset_index()
    tmp.columns = ['year','month','day_of_week','hour','distance_km_round','taxis_per_hour','mean_fare','median_fare','min_fare','max_fare']
    df = pd.merge(df, tmp, how='left', on=['year','month','day_of_week','hour','distance_km_round','taxis_per_hour']) 
    return df

In [ ]:
# Add cost per KM (only works well for the training/test set, not for the Kaggle scoring system)
def add_cost_per_km(df):
    df['cost_per_km'] = df.apply(lambda row: row['distance_km_round'] / row['fare_amount'], axis=1)    
    df['cost_per_km'] = df['cost_per_km'].astype('float32')
    return df

---
# 3. Model Training

## 3.1 Feature Selection

Start by specifying which features should be used:

In [ ]:
# Enable the use of the following features:
use_feature_calendar           = True
use_feature_distance_km        = True
use_feature_distances          = True
use_feature_trip_direction     = True
use_feature_geo_coords_rounded = False
use_feature_geo_cluster        = True
use_feature_geo_diff           = True
use_feature_airports           = True
use_feature_locations          = True
use_feature_hour_statistics    = True

# Bad features, don't use these:
use_feature_ride_statistics    = False
use_feature_fare_statistics    = False
use_feature_cost_per_km        = False

In [ ]:
%%time

### Retrive a clean dataframe from the disk with N rows and add features to it
train_df = get_clean_dataframe(5*1000000) # loads (N * 1M) rows

if (use_feature_calendar):
    train_df = add_calendar(train_df)
    
if (use_feature_hour_statistics):
    train_df = add_hour_statistics(train_df)    

if (use_feature_distance_km):    
    train_df = add_distance_km(train_df)
    
if (use_feature_distances):    
    train_df = add_distances_mht_ecd(train_df)

if (use_feature_trip_direction):
    train_df = add_compass_bearing(train_df)    
    
if (use_feature_geo_coords_rounded):
    train_df = add_geo_coords_rounded(train_df)

if (use_feature_geo_cluster):    
    train_df = add_geo_cluster(train_df)

if (use_feature_geo_diff):    
    train_df = add_geo_diff(train_df)

if (use_feature_airports):
    train_df = add_airports(train_df)

if (use_feature_locations):    
    train_df = add_locations(train_df)
    
if (use_feature_ride_statistics):
    train_df = add_ride_statistics(train_df)

if (use_feature_fare_statistics):
    train_df = add_fare_statistics(train_df)

if (use_feature_cost_per_km):    
    train_df = add_cost_per_km(train_df)
    
display(train_df.head(3))

In [ ]:
import io

# print_mem_usage(): outputs the memory usage from df.info(memory_usage='deep')
def print_mem_usage(df):    
    buf = io.StringIO()
    df.info(buf=buf, memory_usage='deep')
    mem_usage = buf.getvalue().split('\n')[-2]
    print('>>> Memory usage:', mem_usage)

    
# print column names
# print(train_df.columns)
    
# display feature related statistics    
pd.set_option('display.max_columns', 200)       # tells pandas to print all columns (no hiding!)
pd.set_option('display.width', 5000)
pd.set_option('float_format', '{:f}'.format)    # prints the entire number instead of x + ye
display(train_df.describe())

print_mem_usage(train_df)

In [ ]:
# draw the correlation between the features
# train_subset = train_df[:6000]
# sns.pairplot(train_subset, vars=['fare_amount','distance_km_round','lon_diff','lat_diff','geo_cluster','year','month','hour'])
# plt.show()

In [ ]:
# print the correlation of 'fare_amount' with all the other features
print(train_df.corrwith(train_df['fare_amount']))

In [ ]:
# delete features with corr less than 0.1
bad_features_list = ['passenger_count','hour','day','month','day_of_week']
train_df.drop(bad_features_list, axis=1, inplace=True)

## 3.2 Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile, f_regression, RFECV
from sklearn.linear_model import LinearRegression
from itertools import compress

target = train_df[['fare_amount']]
df = train_df.drop(['fare_amount', 'pickup_datetime'], axis=1)

X = df
y = target.values.ravel()

# Split X,y into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print('>>> Train/Test Split done!')

## 3.3 Hyperparameter Tuning (*optional*)

Change `EXEC_GRID_SEARCH` to **True** and be able to run `RandomizedSearchCV()` to find optimal parameters:

In [ ]:
# from scipy.stats import randint as sp_randint
# g = sp_randint(1, 11)
# print(g.rvs(5))

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error

def mean_squared_error_(ground_truth, predictions):
    return math.sqrt(mean_squared_error(ground_truth, predictions))

RMSE = make_scorer(mean_squared_error_, greater_is_better=False)

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

EXEC_GRID_SEARCH = False
print('>>> Perform grid search:', EXEC_GRID_SEARCH)

if (not EXEC_GRID_SEARCH):
#     rfr_model = RandomForestRegressor(n_jobs=7, n_estimators=25, max_features=len(df.columns), max_depth=25, min_samples_split=3, min_samples_leaf=3, random_state=24)
    rfr_model = RandomForestRegressor(n_jobs=-1, n_estimators=35, max_features=len(df.columns), max_depth=28, min_samples_split=4, min_samples_leaf=4, random_state=24)
else:    
    param_grid = {
        'n_estimators'     : sp_randint(8, 45),
        'max_depth'        : sp_randint(3, 20),
        'max_features'     : sp_randint(1, len(df.columns)),                
        'min_samples_split': sp_randint(2, 11),
        'min_samples_leaf' : sp_randint(1, 11),
        'bootstrap'        : [True, False]
    }

    # Estimator for use in random search
    estimator = RandomForestRegressor(random_state=24)

    # Create the random search model
    n_iter_search = 500
    random_search = RandomizedSearchCV(n_jobs=6, estimator=estimator, param_distributions=param_grid, scoring=RMSE, cv=3, n_iter=n_iter_search, verbose=50, random_state=24)
    random_search.fit(X_train, np.log1p(y_train))

    rfr_model = random_search.best_estimator_
    print('>>> Best Score (RMSE):', random_search.best_score_)
    print(f'>>> Best parameters: {random_search.best_params_}')
        
#     report(random_search.cv_results_)

In [ ]:
# view the complete results
if (EXEC_GRID_SEARCH):
    print(random_search.grid_scores_)

## 3.4 Train the Model

In [ ]:
%%time

# Create the model using RandomForestRegressor
rfr_model.fit(X_train, np.log1p(y_train))
print('>>> Model Training done!')

# 4. Model Evaluation

Evaluates the model on the holdout set:

In [ ]:
from lib import gfx  # gfx is another local module under the 'lib' directory
from sklearn import metrics
from math import sqrt

# The prediction is calculated as the arithmetic mean of both model's predictions
# gbr_fare = np.expm1(gbr_model.predict(X_test))  # model 1
rfr_fare = np.expm1(rfr_model.predict(X_test))   # model 2

# y_pred = (gbr_visitors + rfr_visitors) / 2
# y_pred = gbr_fare
y_pred = rfr_fare

rmse_score = sqrt(metrics.mean_squared_error(y_test, y_pred))
print('\n>>> RMSE Score:', rmse_score, '\n') # 2.5M -> 3.6863

# Plot Actual fare data against Predicted fare
gfx.plot_actual_vs_predicted(y, y_test, y_pred)

In [ ]:
features = df.columns[:df.shape[1]]
importances = rfr_model.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,8))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# 5. Submission

## 5.1 Feature Engineering for the Submission dataset

In [ ]:
# Add ride-based features for the test set, copying from the training set
def test_add_ride_statistics(test, train):
    short_train_df = train[['year','month','day','hour','rides_per_hour']]
    tmp_hour = short_train_df.groupby(['year','month','day','hour']).agg({'rides_per_hour' : [np.mean]}).reset_index()
    tmp_hour.columns = ['year','month','day','hour','rides_per_hour']
    test = pd.merge(test, tmp_hour, how='left', on=['year','month','day','hour']) 

    short_train_df = train[['year','month','day','rides_per_day']]
    tmp_day = short_train_df.groupby(['year','month','day']).agg({'rides_per_day' : [np.mean]}).reset_index()
    tmp_day.columns = ['year','month','day','rides_per_day']
    test = pd.merge(test, tmp_day, how='left', on=['year','month','day'])
    
    # At this point, there might be some NaNs laying around    
    test["rides_per_day"].fillna(method='ffill', inplace=True) 
    test["rides_per_hour"].fillna(method='ffill', inplace=True) 
    return test

In [ ]:
# Add mean/median/min/max fare statistics for the test set
def test_add_statistics(test, train):
    # Score 7.82    
#     short_train_df = train[['year','month','day_of_week','hour','distance_km_round','taxis_per_hour','mean_fare','median_fare','min_fare','max_fare']]
#     tmp = short_train_df.groupby(['year','month','day_of_week','hour','distance_km_round','taxis_per_hour']).agg({'mean_fare':[np.mean],'median_fare':[np.mean],'min_fare':[np.mean],'max_fare':[np.mean]}).reset_index()
#     tmp.columns = ['year','month','day_of_week','hour','distance_km_round','taxis_per_hour','mean_fare','median_fare','min_fare','max_fare']    
#     test = pd.merge(test, tmp, how='left', on=['year','month','day_of_week','hour','distance_km_round','taxis_per_hour'])          

    # Score 6.59
    short_train_df = train[['hour','distance_km_round','taxis_per_hour','mean_fare','median_fare','min_fare','max_fare']]
    tmp = short_train_df.groupby(['hour','distance_km_round','taxis_per_hour']).agg({'mean_fare':[np.mean],'median_fare':[np.mean],'min_fare':[np.mean],'max_fare':[np.mean]}).reset_index()
    tmp.columns = ['hour','distance_km_round','taxis_per_hour','mean_fare','median_fare','min_fare','max_fare']
    test = pd.merge(test, tmp, how='left', on=['hour','distance_km_round','taxis_per_hour'])  

    # Score 7.58
#     short_train_df = train[['day_of_week','hour','distance_km_round','taxis_per_hour','mean_fare','median_fare','min_fare','max_fare']]
#     tmp = short_train_df.groupby(['day_of_week','hour','distance_km_round','taxis_per_hour']).agg({'mean_fare':[np.mean],'median_fare':[np.mean],'min_fare':[np.mean],'max_fare':[np.mean]}).reset_index()
#     tmp.columns = ['day_of_week','hour','distance_km_round','taxis_per_hour','mean_fare','median_fare','min_fare','max_fare']
#     test = pd.merge(test, tmp, how='left', on=['day_of_week','hour','distance_km_round','taxis_per_hour'])  
    
    return test

In [ ]:
def test_add_cost_per_km(test):    
    test['cost_per_km'] = test.apply(lambda row: row['distance_km_round'] / row['mean_fare'], axis=1)
    test['cost_per_km'] = test['cost_per_km'].astype('float32')
    return test

## 5.2 Feature Selection for the Submission dataset

In [ ]:
# loads the test dataset used for submission
col_types = {
    'key': 'str',
    'pickup_datetime': 'str', 
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'passenger_count': 'uint8'
}

test_df = pd.read_csv('data/test.csv', usecols=list(col_types.keys()), dtype=col_types)
test_df['pickup_datetime'] = test_df['pickup_datetime'].str.slice(0, 16)
test_df['pickup_datetime'] = pd.to_datetime(test_df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

display(test_df.head(3))
print('>>> test_df shape:', test_df.shape)

In [ ]:
# Add common features
if (use_feature_calendar):
    test_df = add_calendar(test_df)
        
if (use_feature_hour_statistics):
    test_df = add_hour_statistics(test_df)            
        
if (use_feature_distance_km):    
    test_df = add_distance_km(test_df)

if (use_feature_distances):    
    test_df = add_distances_mht_ecd(test_df)

if (use_feature_trip_direction):
    test_df = add_compass_bearing(test_df)        
    
if (use_feature_geo_coords_rounded):    
    test_df = add_geo_coords_rounded(test_df)

if (use_feature_geo_cluster):    
    test_df = add_geo_cluster(test_df, train_df)

if (use_feature_geo_diff):    
    test_df = add_geo_diff(test_df)

if (use_feature_airports):    
    test_df = add_airports(test_df)

if (use_feature_locations):    
    test_df = add_locations(test_df)
    
if (use_feature_ride_statistics):    
    test_df = test_add_ride_statistics(test_df, train_df)

if (use_feature_fare_statistics): 
    test_df = add_fare_statistics(test_df, train_df)        
    # print(test_df.isnull().sum())               # check missing data
    # test_df = test_df.fillna(test_df.mean())    # handle missing data    
        
if (use_feature_cost_per_km): 
    test_df = test_add_cost_per_km(test_df, train_df)         
    
# Remove weak features
test_df.drop(bad_features_list, axis=1, inplace=True)    

display(test_df.head(3))

In [ ]:
# DEBUG: NaNs
# print(test_df.isnull().sum())                             # check missing data
# display(test_df[test_df.isnull().any(axis=1)].head())     # print NaN rows      
# display(test_df.iloc[10:13])                              # print specific rows

display(test_df.describe())
# print(test_df.info())

print_mem_usage(test_df)

In [ ]:
import unittest

L1 = train_df.columns.tolist()
L2 = test_df.columns.tolist()
print('>>> Only *fare_amount* can show up as the difference between both datasets.\n>>> Found:', list(set(L1) - set(L2)))
print('>>> Number of Features for training=', len(L1), 'and testing=', len(L2))

tc = unittest.TestCase('__init__')
tc.assertEqual(len(L1) == len(L2), True)  # test_df includes 'key', so this works

## 5.3 Train & run the final Model on the Submission dataset

This step stores the final results in *submission.csv*:

In [ ]:
%%time

### Select the submission data to be used on the final model
X = train_df.drop(['fare_amount', 'pickup_datetime'], axis=1)
y = train_df[['fare_amount']].values.ravel()

### Create and fit the model
# rfr_model = RandomForestRegressor(n_jobs=7, n_estimators=25, max_features=len(df.columns), max_depth=25, min_samples_split=3, min_samples_leaf=3, random_state=42)
rfr_model = RandomForestRegressor(n_jobs=7, n_estimators=35, max_features=1.0, max_depth=28, min_samples_split=4, min_samples_leaf=4, random_state=42)
rfr_model.fit(X, np.log1p(y))

### Predict fare
X_test = test_df.drop(['key', 'pickup_datetime'], axis=1)
y_pred = np.expm1(rfr_model.predict(X_test))


### Save submission file
submission_df = pd.read_csv('data/sample_submission.csv')
submission_df['fare_amount'] = y_pred
submission_df.to_csv('submission_xgb.csv', index=False)

display(submission_df.head())
print('>>> submission_df shape=', submission_df.shape)
print('>>> File saved sucessfully.')

**Print total notebook runtime for debugging purposes:**

In [ ]:
from datetime import timedelta

elapsed_time = (time.time() - start_time)
print('>>> Runtime:', str(timedelta(seconds=elapsed_time)))